In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout , BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

dataset = 'image_data.csv'
model_save_path = 'HandModel.keras'
tflite_save_path = 'HandModel.tflite'

# Đọc dữ liệu từ file CSV
data = pd.read_csv(dataset)

# Phân tách nhãn và dữ liệu ảnh
labels = data.iloc[:, 0].values
images = data.iloc[:, 1:].values

# Chỉnh sửa nhãn để bắt đầu từ 0 thay vì 1
#labels = labels - 1

# Định hình lại dữ liệu ảnh
images = images.reshape(-1, 28, 28, 1)

# Chuẩn hóa dữ liệu ảnh
#images = images / 255.0

# Chuyển đổi nhãn thành dạng one-hot
labels = to_categorical(labels, num_classes=3)

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Sử dụng ImageDataGenerator để tăng cường dữ liệu
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(X_train)

# Xây dựng mô hình CNN
model = Sequential()
# Giảm số lượng bộ lọc từ 65 xuống 32
model.add(Conv2D(32 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu' , input_shape = (28,28,1)))
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
# Giảm số lượng bộ lọc từ 40 xuống 32
model.add(Conv2D(32 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
# Loại bỏ Dropout hoặc giảm tỷ lệ Dropout
# model.add(Dropout(0.2))
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
# Giảm số lượng bộ lọc từ 25 xuống 16
model.add(Conv2D(16 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Flatten())
# Giảm số lượng units từ 256 xuống 128
model.add(Dense(units = 128 , activation = 'relu'))
# Loại bỏ Dropout hoặc giảm tỷ lệ Dropout
# model.add(Dropout(0.3))
model.add(Dense(units = 3 , activation = 'softmax'))


# Compile mô hình
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Huấn luyện mô hình với dữ liệu tăng cường
model.fit(datagen.flow(X_train, y_train, batch_size=32),
          epochs=10,
          validation_data=(X_test, y_test))

# Đánh giá mô hình trên tập kiểm tra
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f'Test accuracy: {test_acc}')

# Save as a model dedicated to inference
model.save(model_save_path)

# Loading the saved model
#model = tf.keras.models.load_model(model_save_path)

# Inference test
predict_result = model.predict(np.array([X_test[0]]))
print(np.squeeze(predict_result))
print(np.argmax(np.squeeze(predict_result)))

# Tạo một hàm để tạo tập dữ liệu đại diện từ tập huấn luyện
def representative_dataset():
    for data in X_train:
        # Reshape dữ liệu về dạng (1, height, width, channel) và chuyển đổi sang kiểu numpy float32
        yield [data.reshape(1, 28, 28, 1).astype(np.float32)]

# Chuyển đổi mô hình với quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
converter.representative_dataset = representative_dataset  # Thêm representative dataset
tflite_quantized_model = converter.convert()

with open(tflite_save_path, 'wb') as f:
    f.write(tflite_quantized_model)


Epoch 1/10
10/10 [==============================] - 1s 43ms/step - loss: 7.2207 - accuracy: 0.5705 - val_loss: 2.3151 - val_accuracy: 0.5125
Epoch 2/10
10/10 [==============================] - 0s 25ms/step - loss: 0.7752 - accuracy: 0.7962 - val_loss: 0.1116 - val_accuracy: 0.9875
Epoch 3/10
10/10 [==============================] - 0s 29ms/step - loss: 0.1445 - accuracy: 0.9373 - val_loss: 0.0683 - val_accuracy: 0.9750
Epoch 4/10
10/10 [==============================] - 0s 35ms/step - loss: 0.0757 - accuracy: 0.9749 - val_loss: 0.1274 - val_accuracy: 0.9375
Epoch 5/10
10/10 [==============================] - 0s 36ms/step - loss: 0.0795 - accuracy: 0.9592 - val_loss: 0.0211 - val_accuracy: 1.0000
Epoch 6/10
10/10 [==============================] - 0s 37ms/step - loss: 0.0495 - accuracy: 0.9875 - val_loss: 0.0137 - val_accuracy: 1.0000
Epoch 7/10
10/10 [==============================] - 0s 42ms/step - loss: 0.0241 - accuracy: 0.9875 - val_loss: 0.0181 - val_accuracy: 0.9875
Epoch 8/10
10

INFO:tensorflow:Assets written to: C:\Users\tranh\AppData\Local\Temp\tmpy5342dip\assets
c:\Users\tranh\AppData\Local\Programs\Python\Python38\lib\site-packages\tensorflow\lite\python\convert.py:887: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


In [4]:
# Function: Convert some hex value into an array for C programming
def hex_to_c_array(hex_data, var_name):

    c_str = ''

    # Create header guard
    c_str += '#ifndef ' + var_name.upper() + '_H\n'
    c_str += '#define ' + var_name.upper() + '_H\n\n'

    # Add array length at top of file
    c_str += 'const unsigned int ' + var_name + '_len = ' + str(len(hex_data)) + ';\n'

    # Declare C variable
    c_str += 'const unsigned char ' + var_name + '[] = {'
    hex_array = []
    for i, val in enumerate(hex_data):

        # Construct string from hex
        hex_str = format(val, '#04x')

        # Add formatting so each line stays within 80 characters
        if (i + 1) < len(hex_data):
            hex_str += ','
        if (i + 1) % 12 == 0:
            hex_str += '\n '
        hex_array.append(hex_str)

    # Add closing brace
    c_str += '\n ' + format(' '.join(hex_array)) + '\n};\n\n'

    # Close out header guard
    c_str += '#endif //' + var_name.upper() + '_H'

    return c_str

c_model_name = 'HandModel'       # Will be given .h suffix

# Write TFLite model to a C source (or header) file
with open(c_model_name + '.h', 'w') as file:
  file.write(hex_to_c_array(tflite_quantized_model, c_model_name))
